# Lab5.2: LogFileDemo

## Tổng quan bài tập
**Đề bài**: Dự vào dataset, bạn hãy trích xuất các thông tin từ file log đó và chuyển về thành một Dataframe. Dataframe cuối cùng sẽ có Schema như sau:
```
root
 |-- referrer: string (nullable = true)
 |-- count: long (nullable = false)
```
Với trường `referrer` là domain của các trang web và `count` là số lần domain đó xuất hiện trong dataset. Ví dụ:
```
+--------------------------------------+-----+
|referrer                              |count|
+--------------------------------------+-----+
|http://ijavascript.cn                 |1    |
|http://www.google.co.tz               |1    |
|http://www.google.ca                  |6    |
|https://www.google.hr                 |2    |
|https://www.google.ch                 |1    |
+--------------------------------------+-----+
```

Mỗi một dòng ở Dataset sẽ có các thông tin, bạn cần dùng regex để trích xuất các thông tin đó và tổng hợp lại dữ liệu.
Hãy hoàn thiện các phần `[...]` để hoàn thiện đoạn code và giải quyết bài toán trên.

## Tài nguyên tham khảo

Bạn có thể tải tập Dataset tại [link sau](https://drive.google.com/file/d/1Q5PG4-JumQcLwe63J6zLfSRXKjs6o5KC/view?usp=sharing). Sau đó đưa lên Google Drive và kết nối với Colab là có thể sử dụng được. Tập dữ liệu là file `.txt` chứa các dòng log của một hệ thống,

# Cài đặt Spark trên Google Colab

Để có thể sử dụng Spark trên môi trường Google Colab thì bạn sẽ cần cài đặt một số thành phần sau:
- Java 8
- Spark Binary
- findspark

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,333 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,678 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://ppa.launchpadcontent.net/c2d4u.team/c2

Sau đó, bạn sẽ cần khai báo cho hệ thống các đường dẫn cho các thành phần vừa cài.

In [2]:
import os
import findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

findspark.init()

# Kết nối với Google Drive

Để lấy dữ liệu từ các Dataset, bạn sẽ phải lưu file dữ liệu lên Google Drive. Sau đó kết nối Colab đến Google Drive của bạn và lấy được các file dữ liệu.

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# LogFileDemo

Bạn sẽ cần khởi tạo 1 SparkSesson để có thể bắt đầu Spark.

In [4]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract, substring_index

conf = SparkConf() \
    .setMaster('local') \
    .setAppName('LogFileDemo')

spark = SparkSession.builder.config(conf=conf).getOrCreate()
sc = spark.sparkContext

DATASET_PATH = '/content/gdrive/My Drive/DEP303/apache_logs.txt'

Đọc dữ liệu từ Dataset

In [5]:
file_df = spark.read.text(DATASET_PATH)
file_df.printSchema()

root
 |-- value: string (nullable = true)



Trích xuất dữ liệu từ các dòng trong Dataset. `logs_df` sẽ có dạng như sau:
```
+------------+--------------------+--------------------+--------------------+
|          ip|                date|             request|            referrer|
+------------+--------------------+--------------------+--------------------+
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
+------------+--------------------+--------------------+--------------------+
```

In [6]:
log_reg = r'^(\S+) (\S+) (\S+) \[([\w:/]+\s[+\-]\d{4})\] "(\S+) (\S+) (\S+)" (\d{3}) (\S+) "(\S+)" "([^"]*)'

logs_df = file_df.select(regexp_extract('value', log_reg, 1).alias('ip'),
                          regexp_extract('value', log_reg, 4).alias('date'),
                          regexp_extract('value', log_reg, 6).alias('request'),
                          regexp_extract('value', log_reg, 10).alias('referrer'))

logs_df.printSchema()
logs_df.show()

root
 |-- ip: string (nullable = true)
 |-- date: string (nullable = true)
 |-- request: string (nullable = true)
 |-- referrer: string (nullable = true)

+------------+--------------------+--------------------+--------------------+
|          ip|                date|             request|            referrer|
+------------+--------------------+--------------------+--------------------+
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://semicomple...|
|83.149.9.216|17/May/2015:10:05...|/presentations/lo...|http://se

Đếm số lần domain xuất hiện trong file log. Chú ý là bạn phải lọc các trường `referrer` trống khỏi kết quả và chỉ lấy domain của web đó.

In [9]:
logs_df \
    .withColumn("referrer", substring_index("referrer", "/", 3)) \
    .where("trim(referrer) != '-'") \
    .groupBy("referrer") \
    .count() \
    .show(100, truncate=False)

+--------------------------------------+-----+
|referrer                              |count|
+--------------------------------------+-----+
|http://ijavascript.cn                 |1    |
|http://www.google.co.tz               |1    |
|http://www.google.ca                  |6    |
|https://www.google.hr                 |2    |
|https://www.google.ch                 |1    |
|http://www.google.ru                  |6    |
|http://www.raspberrypi-spanish.es     |1    |
|http://semicomplete.com               |2001 |
|http://manpages.ubuntu.com            |2    |
|http://kufli.blogspot.fr              |1    |
|http://www.bing.com                   |6    |
|http://rungie.com                     |1    |
|http://www.google.co.th               |2    |
|https://www.google.cz                 |5    |
|http://danceuniverse.ru               |3    |
|http://www.google.co.uk               |14   |
|http://www.google.rs                  |1    |
|http://kufli.blogspot.in              |1    |
|http://t.co 